In [ ]:
#Connecting to Gdrive
from google.colab import drive 
drive.mount('/content/gdrive')

Mounted at /content/gdrive


In [ ]:
#changing the working directory
%cd /content/gdrive/My Drive/Colab Notebooks/

/content/gdrive/My Drive/Colab Notebooks


In [ ]:
#PART-1
import pandas as pd
import numpy as np
import scipy as sp
import re
import random 

from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from sklearn.naive_bayes import MultinomialNB
from sklearn.linear_model import LogisticRegression
from sklearn.multiclass import OneVsRestClassifier
from sklearn.preprocessing import MultiLabelBinarizer
from sklearn import metrics
from sklearn.metrics import accuracy_score,recall_score,precision_score,f1_score
%matplotlib inline
import nltk
nltk.download('stopwords')
from nltk.corpus import stopwords
from collections import Counter

from sklearn.metrics import accuracy_score
from sklearn.metrics import f1_score
from sklearn.metrics import average_precision_score
from sklearn.metrics import recall_score


#PART-2
import io
import random
import string # to process standard python strings
import numpy as np
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity
import warnings
warnings.filterwarnings('ignore')
import nltk
from nltk.stem import WordNetLemmatizer
nltk.download('popular', quiet=True) # for downloading packages
import json 
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Embedding, GlobalAveragePooling1D
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from sklearn.preprocessing import LabelEncoder

from nltk.chat.util import Chat, reflections
from sklearn.preprocessing import LabelEncoder
import colorama 
colorama.init()
from colorama import Fore, Style, Back
import pickle


[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


In [ ]:
#Supress warnings
import warnings
warnings.filterwarnings('ignore')

# PART-1

# 1.1 Import and analyse the data set.

In [ ]:
#importing the given dataset
Part1_df = pd.read_csv('./NLP1/blogtext.csv')

In [ ]:
## check the shape of the dataset
Part1_df.shape

(681284, 7)

It has nearly 7 lakh records. Its highly impossible to execute on our(student's) machines & lets just use a sample of 25,000 records.

In [ ]:
#First 50k records
Part1_back = Part1_df.copy()
Part1_df = Part1_df.iloc[0:50000]

In [ ]:
#check the shape of the dataset now
Part1_df.shape

(50000, 7)

In [ ]:
#First 5 rows
Part1_df.head(5)

,gender,age,topic,sign,text
0,male,15,Student,Leo,"Info has been found (+/- 100 pages,..."
1,male,15,Student,Leo,These are the team members: Drewe...
2,male,15,Student,Leo,In het kader van kernfusie op aarde...
3,male,15,Student,Leo,testing!!! testing!!!
4,male,33,InvestmentBanking,Aquarius,Thanks to Yahoo!'s Toolbar I can ...


In [ ]:
#Random 5 rows
Part1_df.sample(5)

,gender,age,topic,sign,text
39140,female,23,Education,Capricorn,Today was a heavily-packed-wi...
19447,male,35,Technology,Scorpio,I'm a bus rider. I like not havi...
4972,female,17,indUnk,Scorpio,this shade of red looks more and more l...
47554,female,17,Student,Capricorn,Its Christmas Eve! 9hrs 30mins to g...
32579,female,23,indUnk,Aries,urlLink These is Biz Admin COurse n...


In [ ]:
#Dropping unwanted columns
Part1_df.drop(['id','date'], axis=1, inplace=True)

In [ ]:
#Top-view
Part1_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 50000 entries, 0 to 49999
Data columns (total 5 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   gender  50000 non-null  object
 1   age     50000 non-null  int64 
 2   topic   50000 non-null  object
 3   sign    50000 non-null  object
 4   text    50000 non-null  object
dtypes: int64(1), object(4)
memory usage: 1.9+ MB


- There are no null values. 
- Few datatypes have to be changed

In [ ]:
#Changing the AGE datatype to string object
#Text operations
Part1_df['age'] = Part1_df['age'].astype('str')

In [ ]:
#Check the Age data type again
Part1_df['age'].dtypes

dtype('O')

In [ ]:
#Checking for NULL values explicitly
Part1_df.isna().sum()

gender    0
age       0
topic     0
sign      0
text      0
dtype: int64

There seem to be no null values in the dataset

# 1.2  Perform data pre-processing on the data:
- Data cleansing by removing unwanted characters, spaces, stop words etc Convert text to lowercase.
- Target/label merger and transformation
- Train and test split
- Vectorisation, etc.

In [ ]:
#Text cleaning 

def clean_text(text):
    # remove backslash-apostrophe 
    text = re.sub("\'", "", text) 
    # remove everything except alphabets 
    text = re.sub("[^a-zA-Z]"," ",text) 
    # remove whitespaces 
    text = ' '.join(text.split()) 
    # convert text to lowercase 
    text = text.lower() 
    
    return text

In [ ]:
Part1_df['clean_text'] = Part1_df['text'].apply(lambda x: clean_text(x))

In [ ]:
Part1_df.head(5)

,gender,age,topic,sign,text,clean_text
0,male,15,Student,Leo,"Info has been found (+/- 100 pages,...",info has been found pages and mb of pdf files ...
1,male,15,Student,Leo,These are the team members: Drewe...,these are the team members drewes van der laag...
2,male,15,Student,Leo,In het kader van kernfusie op aarde...,in het kader van kernfusie op aarde maak je ei...
3,male,15,Student,Leo,testing!!! testing!!!,testing testing
4,male,33,InvestmentBanking,Aquarius,Thanks to Yahoo!'s Toolbar I can ...,thanks to yahoo s toolbar i can now capture th...


As we can see, clean text has no extra characters, numbers and upper cases. 


Lets just print one record & see.

In [ ]:
print(Part1_df['text'][0])

           Info has been found (+/- 100 pages, and 4.5 MB of .pdf files) Now i have to wait untill our team leader has processed it and learns html.         


In [ ]:
print(Part1_df['clean_text'][0])

info has been found pages and mb of pdf files now i have to wait untill our team leader has processed it and learns html


In [ ]:
## Getting stop words from english language 
stop_words = set(stopwords.words('english'))

In [ ]:
# Function to remove stopwords
def remove_stopwords(text):
    no_stopword_text = [w for w in text.split() if not w in stop_words]
    return ' '.join(no_stopword_text)

In [ ]:
#calling the stop words function
Part1_df['clean_text'] = Part1_df['clean_text'].apply(lambda x: remove_stopwords(x))

In [ ]:
#Before removing stop words: the sentence is :
#info has been found pages and mb of pdf files now i have to wait untill our team leader has processed it and learns html
print(Part1_df['clean_text'][0])

info found pages mb pdf files wait untill team leader processed learns html


As we can see above, all the stop words in the sentence such as  has/been/and/the/of/now..etc are removed.



###### Target/label merger and transformation

In [ ]:
#Declaring the label columns
Part1_df['labels']=' '

In [ ]:
# Lets extract the required key labels into one single column
# Multi label classification
for w in range(0,Part1_df.shape[0]):
    blog_label=[]
    blog_label.append(Part1_df['gender'][w])
    blog_label.append(Part1_df['age'][w])
    blog_label.append(Part1_df['topic'][w])
    blog_label.append(Part1_df['sign'][w])
    Part1_df['labels'][w]=blog_label

In [ ]:
#Lets just print the head
Part1_df.head()

,gender,age,topic,sign,text,clean_text,labels
0,male,15,Student,Leo,"Info has been found (+/- 100 pages,...",info found pages mb pdf files wait untill team...,"[male, 15, Student, Leo]"
1,male,15,Student,Leo,These are the team members: Drewe...,team members drewes van der laag urllink mail ...,"[male, 15, Student, Leo]"
2,male,15,Student,Leo,In het kader van kernfusie op aarde...,het kader van kernfusie op aarde maak je eigen...,"[male, 15, Student, Leo]"
3,male,15,Student,Leo,testing!!! testing!!!,testing testing,"[male, 15, Student, Leo]"
4,male,33,InvestmentBanking,Aquarius,Thanks to Yahoo!'s Toolbar I can ...,thanks yahoo toolbar capture urls popups means...,"[male, 33, InvestmentBanking, Aquarius]"


Cool, so since we merged Gender, Age, topic, sign into Labels. Lets drop these columns

In [ ]:
#Part1_df = Part1_df.drop(['gender','age','topic','sign'],axis=1)

Part1_df = Part1_df[['clean_text','labels']]
#Final dataset needed for the model
Part1_df.head()

,clean_text,labels
0,info found pages mb pdf files wait untill team...,"[male, 15, Student, Leo]"
1,team members drewes van der laag urllink mail ...,"[male, 15, Student, Leo]"
2,het kader van kernfusie op aarde maak je eigen...,"[male, 15, Student, Leo]"
3,testing testing,"[male, 15, Student, Leo]"
4,thanks yahoo toolbar capture urls popups means...,"[male, 33, InvestmentBanking, Aquarius]"


In [ ]:
Part1_df['labels'].values

array([list(['male', '15', 'Student', 'Leo']),
       list(['male', '15', 'Student', 'Leo']),
       list(['male', '15', 'Student', 'Leo']), ...,
       list(['male', '23', 'Advertising', 'Taurus']),
       list(['male', '23', 'Advertising', 'Taurus']),
       list(['male', '23', 'Advertising', 'Taurus'])], dtype=object)

#### Train and test split

In [ ]:
## Divide the features and labels into X and Y and split is done into train and test 
X = Part1_df['clean_text']
y = Part1_df['labels']

In [ ]:
# Split dataset into training and test set before creating dtm using count vectorizer 
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=7)

In [ ]:
print(X_train.shape)
print(y_train.shape)
print(X_test.shape)
print(y_test.shape)

(35000,)
(35000,)
(15000,)
(15000,)


In [ ]:
X_test

29430    urllink wish could work friends landlord full ...
27750    actually soph agree thats move page long roman...
47782                                        welcome world
10498    time discuss something architectural due recen...
24747    running race really lapsed today spent cumulat...
                               ...                        
18878    received e mail minute ago former president li...
44229    urllink eisenhower national historic site nati...
41470    saw rej today like wow didnt know much missed ...
36187    anybody else think elevated stands use gerunds...
14571                                                 hmmm
Name: clean_text, Length: 15000, dtype: object

In [ ]:
y_test

29430           [male, 24, Chemicals, Libra]
27750        [male, 16, indUnk, Sagittarius]
47782       [female, 17, Student, Capricorn]
10498       [male, 26, Architecture, Cancer]
24747              [male, 24, indUnk, Aries]
                        ...                 
18878            [male, 26, Student, Pisces]
44229        [male, 34, Education, Aquarius]
41470          [female, 17, Fashion, Taurus]
36187    [male, 23, Non-Profit, Sagittarius]
14571              [male, 23, Internet, Leo]
Name: labels, Length: 15000, dtype: object

#### Vectorisation

In [ ]:
# Creating bag of words which include 1-grams and 2-gram
cvect = CountVectorizer(ngram_range=(1,2))

In [ ]:
# learn training data vocabulary, then use it to create a document-term matrix
cvect.fit(X_train)

X_train_dtm = cvect.transform(X_train)

In [ ]:
#Check the vocablury size
len(cvect.vocabulary_)

2218958

In [ ]:
cvect.get_feature_names()[:15]

['aa',
 'aa aa',
 'aa advert',
 'aa amazing',
 'aa batteries',
 'aa class',
 'aa ended',
 'aa eriol',
 'aa flights',
 'aa forms',
 'aa going',
 'aa great',
 'aa haha',
 'aa htm',
 'aa join']

In [ ]:
# examin & print the document-term matrix
X_train_dtm

<35000x2218958 sparse matrix of type '<class 'numpy.int64'>'
	with 6249872 stored elements in Compressed Sparse Row format>

In [ ]:
print(X_train_dtm[:5])

  (0, 40750)	1
  (0, 40793)	1
  (0, 414724)	1
  (0, 414847)	1
  (0, 679520)	1
  (0, 681165)	1
  (0, 770318)	1
  (0, 770395)	1
  (0, 1188664)	1
  (0, 1189008)	1
  (0, 1574916)	1
  (0, 1574923)	1
  (0, 1672598)	1
  (0, 1672714)	1
  (0, 1744070)	1
  (0, 1744099)	1
  (0, 2049174)	1
  (0, 2049295)	1
  (0, 2079282)	1
  (0, 2079435)	1
  (0, 2102988)	1
  (1, 78294)	1
  (1, 78748)	1
  (1, 79991)	1
  (1, 80371)	1
  :	:
  (4, 1034665)	1
  (4, 1092536)	1
  (4, 1099821)	1
  (4, 1104794)	1
  (4, 1104973)	1
  (4, 1199078)	1
  (4, 1199107)	1
  (4, 1319506)	1
  (4, 1319700)	1
  (4, 1364766)	1
  (4, 1364869)	1
  (4, 1448298)	1
  (4, 1448325)	1
  (4, 1476538)	1
  (4, 1477429)	1
  (4, 1542103)	1
  (4, 1542143)	1
  (4, 1878360)	1
  (4, 1878372)	1
  (4, 1895917)	1
  (4, 1896455)	1
  (4, 2049174)	1
  (4, 2049507)	1
  (4, 2140372)	1
  (4, 2140428)	1


In [ ]:
X_test_dtm = cvect.transform(X_test)

In [ ]:
# examine & print the document-term matrix
X_test_dtm

<15000x2218958 sparse matrix of type '<class 'numpy.int64'>'
	with 1920675 stored elements in Compressed Sparse Row format>

In [ ]:
print(X_test_dtm[:5])

  (0, 29020)	1
  (0, 85020)	2
  (0, 98461)	5
  (0, 98555)	1
  (0, 98959)	1
  (0, 99986)	1
  (0, 100998)	1
  (0, 101021)	1
  (0, 101878)	1
  (0, 106437)	1
  (0, 106496)	1
  (0, 110655)	1
  (0, 111343)	1
  (0, 127374)	1
  (0, 133712)	2
  (0, 137465)	1
  (0, 138939)	1
  (0, 139020)	1
  (0, 145206)	1
  (0, 145603)	1
  (0, 164827)	1
  (0, 170148)	1
  (0, 170610)	1
  (0, 217746)	1
  (0, 226438)	1
  :	:
  (4, 1803617)	1
  (4, 1825330)	1
  (4, 1875062)	1
  (4, 1875115)	1
  (4, 1914022)	1
  (4, 1935941)	1
  (4, 1938089)	1
  (4, 1938573)	1
  (4, 1941356)	1
  (4, 1959921)	1
  (4, 1961494)	1
  (4, 1971178)	1
  (4, 1973482)	1
  (4, 2000600)	1
  (4, 2046351)	1
  (4, 2049011)	1
  (4, 2068712)	1
  (4, 2074376)	1
  (4, 2077910)	1
  (4, 2110914)	1
  (4, 2111670)	1
  (4, 2111767)	2
  (4, 2113361)	1
  (4, 2114020)	1
  (4, 2158758)	1


In [ ]:
print(X_train_dtm.shape)
print(X_test_dtm.shape)

(35000, 2218958)
(15000, 2218958)


Create a dictionary to get the count of every label i.e. the key will be label name and value will be the total count of the label

In [ ]:
label_counts=dict()

for labels in Part1_df.labels.values:
    for label in labels:
        if label in label_counts:
            label_counts[str(label)]+=1
        else:
            label_counts[str(label)]=1

In [ ]:
label_counts

{'13': 745,
 '14': 2043,
 '15': 3508,
 '16': 4156,
 '17': 6859,
 '23': 5518,
 '24': 5746,
 '25': 2837,
 '26': 2869,
 '27': 4094,
 '33': 1654,
 '34': 1886,
 '35': 3365,
 '36': 1985,
 '37': 310,
 '38': 196,
 '39': 412,
 '40': 192,
 '41': 394,
 '42': 96,
 '43': 150,
 '44': 38,
 '45': 93,
 '46': 330,
 '47': 206,
 '48': 318,
 'Accounting': 364,
 'Advertising': 273,
 'Agriculture': 78,
 'Aquarius': 4784,
 'Architecture': 70,
 'Aries': 7795,
 'Arts': 1817,
 'Automotive': 116,
 'Banking': 283,
 'Biotech': 101,
 'BusinessServices': 416,
 'Cancer': 4589,
 'Capricorn': 3819,
 'Chemicals': 75,
 'Communications-Media': 1603,
 'Construction': 28,
 'Consulting': 243,
 'Education': 2646,
 'Engineering': 1402,
 'Environment': 6,
 'Fashion': 1805,
 'Gemini': 2558,
 'Government': 599,
 'HumanResources': 79,
 'Internet': 1420,
 'InvestmentBanking': 85,
 'Law': 308,
 'LawEnforcement-Security': 125,
 'Leo': 3904,
 'Libra': 4378,
 'Manufacturing': 441,
 'Maritime': 54,
 'Marketing': 414,
 'Military': 194,
 '

#### Transform the labels

We need to transform labels in a binary form and the prediction will be a mask of 0s and 1s. For this purpose, it is convenient to convert your train and test labels using MultiLabelBinarizer

In [ ]:
from sklearn.preprocessing import MultiLabelBinarizer
binarizer=MultiLabelBinarizer(classes=sorted(label_counts.keys()))

In [ ]:
y_train = binarizer.fit_transform(y_train)
y_test = binarizer.transform(y_test)
y_test

array([[0, 0, 0, ..., 0, 0, 1],
       [0, 0, 0, ..., 0, 1, 1],
       [0, 0, 0, ..., 1, 0, 0],
       ...,
       [0, 0, 0, ..., 1, 0, 0],
       [0, 0, 0, ..., 0, 0, 1],
       [0, 0, 0, ..., 0, 0, 1]])

#1.3. Design, train, tune and test the best text classifier.

Let's use the One-vs-Rest approach, which is implemented in OneVsRestClassifier class. This is the most commonly used strategy for multiclass classification. In this approach k classifiers (= number of tags) are trained. As a basic classifier, let's use LogisticRegression.

In [ ]:
model = LogisticRegression(solver='lbfgs', max_iter=100)
model = OneVsRestClassifier(model)
model.fit(X_train_dtm,y_train)

OneVsRestClassifier(estimator=LogisticRegression())

In [ ]:
## make predictions on test set
y_pred_test = model.predict(X_test_dtm)

In [ ]:
y_pred_test

array([[0, 0, 0, ..., 1, 0, 0],
       [0, 0, 0, ..., 0, 1, 1],
       [0, 0, 0, ..., 0, 0, 1],
       ...,
       [0, 0, 0, ..., 0, 0, 1],
       [0, 0, 0, ..., 1, 0, 0],
       [0, 0, 0, ..., 0, 0, 1]])

# 1.4. Display and explain detail the classification report

Classification Report explains below metrics.

 1. **Accuracy score**
      * a ratio of correctly predicted observation to the total observations.
 2. **F1 Score**
      * F1 Score is the weighted average of Precision and Recall
 3. Average precision score.
      * Macro averaged precision: calculate precision for all classes individually and then average them.
      * Micro averaged precision: calculate class wise true positive and false positive and then use that to calculate overall precision.

 4. Average recall score


In genaral, 

**Precision** : the ratio of correctly predicted positive observations to the total predicted positive observations.

**Recall** : the ratio of correctly predicted positive observations to the all observations in actual class - yes.

In [ ]:
## evaluating model performance via accuracy score
accuracy_score(y_test, y_pred_test)

0.1368

In [ ]:
## evaluating performance of the model using f1 score 
f1_score(y_test, y_pred_test,average='micro')

0.5099382689024123

In [ ]:
#Lets define the functions through which we can explain in detail

def display_metrics_micro(Ytest, Ypred):
    print('Accuracy score: ', accuracy_score(Ytest, Ypred))
    print('F1 score: Micro', f1_score(Ytest, Ypred, average='micro'))
    print('Average precision score: Micro', average_precision_score(Ytest, Ypred, average='micro'))
    print('Average recall score: Micro', recall_score(Ytest, Ypred, average='micro'))
    
    
def display_metrics_macro(Ytest, Ypred):
    print('Accuracy score: ', accuracy_score(Ytest, Ypred))
    print('F1 score: Macro', f1_score(Ytest, Ypred, average='macro'))
    print('Average recall score: MAcro', recall_score(Ytest, Ypred, average='macro'))
    
def display_metrics_weighted(Ytest, Ypred):
    print('Accuracy score: ', accuracy_score(Ytest, Ypred))
    print('F1 score: weighted', f1_score(Ytest, Ypred, average='weighted'))
    print('Average precision score: weighted', average_precision_score(Ytest, Ypred, average='weighted'))
    print('Average recall score: weighted', recall_score(Ytest, Ypred, average='weighted'))

In [ ]:
display_metrics_micro(y_test,y_pred_test)

Accuracy score:  0.1368
F1 score: Micro 0.5099382689024123
Average precision score: Micro 0.3189526240275475
Average recall score: Micro 0.38825


In [ ]:
display_metrics_macro(y_test,y_pred_test)

Accuracy score:  0.1368
F1 score: Macro 0.26203586538724494
Average recall score: MAcro 0.18032851721552834


In [ ]:
display_metrics_weighted(y_test,y_pred_test)

Accuracy score:  0.1368
F1 score: weighted 0.4697518841901344
Average precision score: weighted 0.3806597734444129
Average recall score: weighted 0.38825


Few observations here:

- Metrics are not the best we expected it to be.
- Our model is built on just 50k records, which is just less than 10% of our original dataset. This could a reason for underfitting.
- The generality of multi-label problems inevitably makes it more difficult to learn. 
- An intuitive approach to solving multi-label problem is to decompose it into multiple independent binary classification problems (one per category).

- In an “one-to-rest” strategy, one could build multiple independent classifiers and, for an unseen instance, choose the class for which the confidence is maximized.

- The main assumption here is that the labels are mutually exclusive. You do not consider any underlying correlation between the classes in this method.

#1.5. Print the true vs predicted labels for any 5 entries from the dataset.

In [ ]:
y_test[0:5]

array([[0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1],
       [0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1],
       [0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 1, 0, 0],
       [0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0

In [ ]:
y_pred_test[0:5]

array([[0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0],
       [0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1],
       [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1],
       [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0

In [ ]:
#Lets just walk through our predictions in detail
def print_predicted(y_predicted, y_test = y_test , n = 5):
    j = []
    for i in range(n):
        j.append(random.randint(0, len(y_test)))
    print(j)
                 
    for k in j:
        print(binarizer.inverse_transform(y_predicted)[k])
        print(binarizer.inverse_transform(y_test)[k])
        print("--------------------------------")

In [ ]:
print_predicted(y_predicted=y_pred_test,y_test=y_test, n= 10)

[8106, 2050, 7637, 9084, 6669, 6310, 5325, 13386, 12868, 2223]
('35', 'Aries', 'Technology', 'male')
('35', 'Aries', 'Technology', 'male')
--------------------------------
('male',)
('27', 'Aquarius', 'Religion', 'male')
--------------------------------
('male',)
('26', 'Pisces', 'Student', 'male')
--------------------------------
('male',)
('48', 'Aries', 'Communications-Media', 'male')
--------------------------------
('female',)
('35', 'Libra', 'female', 'indUnk')
--------------------------------
('Leo', 'female')
('46', 'Leo', 'female', 'indUnk')
--------------------------------
('female',)
('27', 'Aries', 'Communications-Media', 'female')
--------------------------------
('male',)
('17', 'Non-Profit', 'Sagittarius', 'male')
--------------------------------
('17', 'Aquarius', 'female', 'indUnk')
('17', 'Aquarius', 'female', 'indUnk')
--------------------------------
('female', 'indUnk')
('16', 'Libra', 'female', 'indUnk')
--------------------------------


# PART-2

Design a python based interactive semi - rule based chatbot which can do the following:

1. Start chat session with greetings and ask what the user is looking for.
2. Accept dynamic text based questions from the user. Reply back with relevant answer from the designed corpus.
3. End the chat session only if the user requests to end else ask what the user is looking for. Loop continues till the user asks to end it.

In [ ]:
!pip install colorama

Let's read the given json file contents here.

In [ ]:
with open('./NLP1/GL Bot.json') as file:
    data = json.load(file)

Given json file has,

- Tags - topic/subject of input would be.
- patterns - the input query patterns
- responses - respective single line response, that should be from the bot.
- context set as null.

In [ ]:
training_sentences = []
training_labels = []
labels = []
responses = []


for intent in data['intents']:
    for pattern in intent['patterns']:
        training_sentences.append(pattern)
        training_labels.append(intent['tag'])
    responses.append(intent['responses'])
    
    if intent['tag'] not in labels:
        labels.append(intent['tag'])
        
num_classes = len(labels)

In [ ]:
#Label Encoding
lbl_encoder = LabelEncoder()
lbl_encoder.fit(training_labels)
training_labels = lbl_encoder.transform(training_labels)

In [ ]:
#Tokenization
vocab_size = 1000
embedding_dim = 16
max_len = 20
oov_token = "<OOV>"

tokenizer = Tokenizer(num_words=vocab_size, oov_token=oov_token)
tokenizer.fit_on_texts(training_sentences)
word_index = tokenizer.word_index
sequences = tokenizer.texts_to_sequences(training_sentences)
padded_sequences = pad_sequences(sequences, truncating='post', maxlen=max_len)

In [ ]:
#Building the ML model
model = Sequential()
model.add(Embedding(vocab_size, embedding_dim, input_length=max_len))
model.add(GlobalAveragePooling1D())
model.add(Dense(16, activation='relu'))
model.add(Dense(16, activation='relu'))
model.add(Dense(num_classes, activation='softmax'))

model.compile(loss='sparse_categorical_crossentropy', 
              optimizer='adam', metrics=['accuracy'])

model.summary()
epochs = 500
history = model.fit(padded_sequences, np.array(training_labels), epochs=epochs)

Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding_1 (Embedding)     (None, 20, 16)            16000     
                                                                 
 global_average_pooling1d_1   (None, 16)               0         
 (GlobalAveragePooling1D)                                        
                                                                 
 dense_3 (Dense)             (None, 16)                272       
                                                                 
 dense_4 (Dense)             (None, 16)                272       
                                                                 
 dense_5 (Dense)             (None, 8)                 136       
                                                                 
Total params: 16,680
Trainable params: 16,680
Non-trainable params: 0
__________________________________________________

In [ ]:
# to save the trained model
model.save("./NLP1/chat_model")

# to save the fitted tokenizer
with open('tokenizer.pickle', 'wb') as handle:
    pickle.dump(tokenizer, handle, protocol=pickle.HIGHEST_PROTOCOL)
    
# to save the fitted label encoder
with open('label_encoder.pickle', 'wb') as ecn_file:
    pickle.dump(lbl_encoder, ecn_file, protocol=pickle.HIGHEST_PROTOCOL)

INFO:tensorflow:Assets written to: ./NLP1/chat_model/assets


In [ ]:
with open("./NLP1/GL Bot.json") as file:
    data = json.load(file)


def chat():
    print("Chat with the BOT (type: stop/quit)")
    print("If answer is not right (type: *)")
    # load trained model
    model = keras.models.load_model('./NLP1/chat_model')

    # load tokenizer object
    with open('tokenizer.pickle', 'rb') as handle:
        tokenizer = pickle.load(handle)

    # load label encoder object
    with open('label_encoder.pickle', 'rb') as enc:
        lbl_encoder = pickle.load(enc)

    # parameters
    max_len = 20
    
    while True:
        print(Fore.LIGHTBLUE_EX + "You: " + Style.RESET_ALL, end="")
        inp = input()
        if inp.lower() == "*":
          print("BOT: Please rephrase your question and try again")
        if (inp.lower() == "quit") or (inp.lower() == "stop") :
            break
   
        result = model.predict(keras.preprocessing.sequence.pad_sequences(tokenizer.texts_to_sequences([inp]),
                                             truncating='post', maxlen=max_len))
        tag = lbl_encoder.inverse_transform([np.argmax(result)])

        for i in data['intents']:
            if i['tag'] == tag:
                print(Fore.GREEN + "THE BOT:" + Style.RESET_ALL , np.random.choice(i['responses']))

        



print(Fore.YELLOW + "Start messaging with the bot (type quit to stop)!" + Style.RESET_ALL)
chat()

Start messaging with the bot (type quit to stop)!
Chat with the BOT (type: stop/quit)
If answer is not right (type: *)
You: hi
THE BOT: Hello! how can i help you ?
You: yo
THE BOT: Hello! how can i help you ?
You: sgd
THE BOT: Link: Neural Nets wiki
You: NN
THE BOT: Hello! how can i help you ?
You: ML
THE BOT: Link: Machine Learning wiki 
You: olympus
THE BOT: Link: Olympus wiki
You: nb
THE BOT: Link: Machine Learning wiki 
You: sl
THE BOT: Link: Machine Learning wiki 
You: later
THE BOT: I hope I was able to assist you, Good Bye
You: hell
THE BOT: Tarnsferring the request to your PM
You: not solved
THE BOT: I hope I was able to assist you, Good Bye
You: my problem is not solved
THE BOT: Tarnsferring the request to your PM
You: jerk
THE BOT: Please use respectful words
You: name please
THE BOT: I am your virtual learning assistant
You: later
THE BOT: I hope I was able to assist you, Good Bye
You: stop


============================== END of the Notebook =============================